In [1]:
import multiprocessing as mp
import math
import numpy as np
import time
import struct
import sys


In [ ]:
M_PI = 3.14159265358979323846
DEG_CIRCLE = 360
DEG_TO_RAD = (M_PI / (DEG_CIRCLE / 2))
RAD_TO_DEG = ((DEG_CIRCLE / 2) / M_PI)

flightWidth = 670
latBoxSize = 25
intsThreshold = 240
rngDistance = 30
llTolerance = 2
dblPctWidth = ((flightWidth / 2) * .25)
rngThreshold = (intsThreshold - (intsThreshold * .05))

# Square root of lattice box size
sqrLatBS3 = math.sqrt(latBoxSize)
# Third of size of each lattice box
latBS3 = sqrLatBS3 / 3
# Count along the X lines
cntX = int(flightWidth / latBS3)

blnSlope = None
latXOffset = latYOffset = 0
cntYlat = 0
centX = 0
centY = 0
theta = 0

# x, y, z, scan_direction, intensity, raw_time
las = np.load('data/LiDAR.npy')

def rotate(centerX, centerY, inputX, inputY, theta_v):
    cx = inputX - centerX
    cy = inputY - centerY

    a = math.atan(cy / cx)

    if cx < 0:
        a = a + M_PI

    h = math.sqrt(abs(cx * cx + cy * cy))

    dx = math.cos(a + theta_v) * h
    dy = math.sin(a + theta_v) * h

    rStartX = centerX + dx
    rStartY = centerY + dy

    return rStartX, rStartY

def runDataCheck(queue, pid, rStart, rEnd):
    intArray = np.empty([(rEnd - rStart),2], dtype='d')
    start = time.perf_counter()
    counter = 0
    cntSuspect = 0
    intFirst = 0
    intSecond = 0
    intThird = 0
    intFourth = 0

    for ps in range(rStart, rEnd):
        pnt = las[ps]

        if blnSlope:
            lasRPointX, lasRPointY = rotate(centX, centY, pnt[0], pnt[1], theta)
        else:
            lasRPointX, lasRPointY = rotate(centX, centY, pnt[0], pnt[1], -theta)

        dblDistance = abs(lasRPointX - centX)

        if dblDistance > (flightWidth / 4) - dblPctWidth:
            newrow = [pnt[0], pnt[1]]
            intArray[counter] = newrow
            counter = counter + 1

        intFlag = 0

        # find the difference between the LIDAR points and the Lattice Offsets
        offsetDiffX = lasRPointX - latXOffset
        if offsetDiffX > flightWidth:
            continue

        offsetDiffY = lasRPointY - latYOffset

        # offsetDiffX / latBS3 should produce how many latBS3 cells there are.
        # Then multiple this number by the value of latBS3 and add it to the layYOffset1.
        offsetX = latXOffset + (latBS3 * (offsetDiffX / latBS3))

        # offsetDiffY / latBS3 should produce how many latBS3 cells there are.
        # Take the number of cells and multiple by the number of X cells for each Y.
        # Then multiple this number by the value of latBS3 and add it to the layYOffset1.
        offsetY = latYOffset + (latBS3 * (offsetDiffY / latBS3))

        # Calculate the position of X/Y in the lattice.
        # Subtract 1 from the count, since the lattice is 0 based.
        cntLattice = int((cntX * (offsetDiffY / latBS3)) + (offsetDiffX / latBS3) - 1)

        if cntLattice > (cntYlat * cntX):
            continue

        if pnt[2] > (zAvg - rngDistance):
            lattice[cntLattice] = 5  # point outside of AOI, exceeded threshold, or z-axis to close to aircraft
            continue

        if pnt[4] > intsThreshold:
            lattice[cntLattice] = 5  # point outside of AOI, exceeded threshold, or z-axis to close to aircraft
            continue

        # If lidar point is outside lattice, then skip
        if (latXOffset > lasRPointX) or (latYOffset > lasRPointY):
            lattice[cntLattice] = 5  # point outside of AOI, exceeded threshold, or z-axis to close to aircraft
            continue

        if (pnt[4] > rngThreshold) and (pnt[4] < intsThreshold):
            intFlag = 1  # suspect data point

        # Set value to first point to check
        fltPointChk = ((lasRPointX - offsetX) ** 2) + ((lasRPointY - offsetY) ** 2)

        # Check to see if fltPointChk is within the radius of first point
        if fltPointChk <= pointRadius:
            # Inside or on radius of the circle
            if intFlag == 1:
                lattice[cntLattice] = 3  # suspect
                cntSuspect = cntSuspect + 1
            else:
                # This is a good data point, so set the lattice point to good
                if lattice[cntLattice] == 3:
                    cntSuspect = cntSuspect - 1
                lattice[cntLattice] = 1  # good
            intFirst += 1

        # Set value to second point to check
        fltPointChk = ((lasRPointX - (offsetX + latBS3)) ** 2) + ((lasRPointY - offsetY) ** 2)

        # Check to see if fltPointChk is within the radius of second point
        if (fltPointChk <= pointRadius) and ((cntLattice + 1) < (cntYlat * cntX)):
            # Inside or on radius of the circle
            if intFlag == 1:
                lattice[cntLattice + 1] = 3  # suspect
                cntSuspect = cntSuspect + 1
            else:
                # This is a good data point, so set the lattice point to good
                if lattice[cntLattice + 1] == 3:
                    cntSuspect = cntSuspect - 1
                lattice[cntLattice + 1] = 1  # good
            intSecond += 1

        # Set value to third point to check
        fltPointChk = pow((lasRPointX - offsetX), 2) + pow(lasRPointY - (offsetY + 1), 2)

        # Check to see if fltPointChk is within the radius of third point
        if (fltPointChk <= pointRadius) and ((cntLattice + cntX) < (cntYlat * cntX)):
            # Inside or on radius of the circle
            if intFlag == 1:
                lattice[cntLattice + cntX] = 3  # suspect
                cntSuspect = cntSuspect + 1
            else:
                # This is a good data point, so set the lattice point to good
                if lattice[cntLattice + cntX] == 3:
                    cntSuspect = cntSuspect - 1
                lattice[cntLattice + cntX] = 1  # good
            intThird += 1

        # Set value to fourth point to check
        fltPointChk = pow((lasRPointX - (offsetX + latBS3)), 2) + pow(lasRPointY - (offsetY + 1), 2)

        # Check to see if fltPointChk is within the radius of fourth point
        if (fltPointChk <= pointRadius) and ((cntLattice + cntX + 1) < (cntYlat * cntX)):
            # Inside or on radius of the circle
            if intFlag == 1:
                lattice[cntLattice + cntX + 1] = 3  # suspect
                cntSuspect = cntSuspect + 1
            else:
                # This is a good data point, so set the lattice point to good
                if lattice[cntLattice + cntX + 1] == 3:
                    cntSuspect = cntSuspect - 1
                lattice[cntLattice + cntX + 1] = 1  # good
            intFourth += 1

    end = time.perf_counter()
    print(pid, ' - ', f'Finished in {round(end - start, 6)} second(s)')

    queue.put(intArray)
    return




if __name__ == '__main__':
    # ###############
    # Initial run data
    # ###############
    start = time.perf_counter()


    print('Flight Width              :', flightWidth)
    print('Lattice Box Size          :', latBoxSize)
    print('Intensity Threshold       :', intsThreshold)
    print('Range Distance            :', rngDistance)
    print('Point Reduction Tolerance :', llTolerance)
    print('sqrLatBS3                 :', sqrLatBS3)
    print('latBS3                    :', latBS3)
    print('cntX                      :', cntX)

    end = time.perf_counter()
    print(f'Finished in {round(end - start, 6)} second(s)')
    print('')

    # ====================================================================================
    # ###############
    # Pull LAS Header Information
    # ###############
    start = time.perf_counter()

    # 0     1     2     3     4     5     6
    # MaxX, MaxY, MaxZ, MinX, MinY, MinZ, pnt_rcrd_cnt
    lasHdr = np.load('data/LiDAR_Hdr.npy')

    las = lasHdr[0]

    lasMaxPointX = las[0]
    lasMaxPointY = las[1]
    lasMaxPointZ = las[2]
    lasMinPointX = las[3]
    lasMinPointY = las[4]
    lasMinPointZ = las[5]
    lasPointCount = int(las[6])

    intRecCnt = lasPointCount - 1

    print('min x y z                 : {} {} {}'.format(lasMinPointX, lasMinPointY, lasMinPointZ))
    print('max x y z                 : {} {} {}'.format(lasMaxPointX, lasMaxPointY, lasMaxPointZ))

    # 0  1  2  3                    4          5
    # X, Y, Z, scan_direction_flag, intensity, gps_time
    las = np.load('data/LiDAR.npy')

    # Find LAS first time using scan direction
    tmpDir = las[0][3]

    for px in range(0, intRecCnt):
        pnt = las[px]
        if((pnt[3] != tmpDir) and (pnt[5] > 0)):
            if(px > 1):
                pnt = las[px - 1]
                lasFirstTime = pnt[5]
                print('Las First Time            :', lasFirstTime)
                break

    # Find LAS last time using scan direction
    tmpDir = las[intRecCnt][3]

    for px in reversed(range(0, intRecCnt - 1)):
        pnt = las[px]
        if((pnt[3] != tmpDir) and (pnt[5] > 0)):
            if(px < intRecCnt):
                p = las[px - 1]
                lasLastTime = pnt[5]
                print('LAS Last Time             :', lasLastTime)
                break

    end = time.perf_counter()
    print(f'Finished in {round(end - start, 6)} second(s)')
    print('')

    # ====================================================================================
    # ###############
    # Pull Trajectory Header Information
    # ###############
    start = time.perf_counter()

    # define trajectory header structure
    hdrstruct = '8s4i79sB2d2i400s2d400s2d'
    header = struct.Struct(hdrstruct)

    # define structure for trajectory position
    trjstruct = '7d2i'
    position = struct.Struct(trjstruct)

    with open('data/414299_414596.trj', "rb") as file:
        traj_hdr = header.unpack(file.read(header.size))
        # Move file pointer to before points
        file.seek(traj_hdr[2])

        poscnt = traj_hdr[3]
        print('Position Count            :', poscnt)

        trjpos = np.empty([poscnt, 9], dtype='d')
        for pos in range(poscnt - 1):
            trjpos[pos] = np.array(position.unpack(file.read(position.size)))

        zAvg = 0
        trjFirstFound = 0
        trjLastFound = 0

        for pos in range(poscnt - 1):
            zAvg += trjpos[pos, 3]
            if (trjFirstFound == 0) and (trjpos[pos, 0] > lasFirstTime):
                trjStartX = trjpos[pos, 1]
                trjStartY = trjpos[pos, 2]
                trjFirstFound = 1
            if (trjLastFound == 0) and (trjpos[pos, 0] > lasLastTime):
                trjEndX = trjpos[pos, 1]
                trjEndY = trjpos[pos, 2]
                trjLastFound = 1

    print('')
    print('Average Z for aircraft from Trajectory file is {}'.format(zAvg / poscnt))
    print('Traj Start X              :', trjStartX)
    print('Traj Start Y              :', trjStartY)
    print('Traj End X                :', trjEndX)
    print('Traj End Y                :', trjEndY)

    end = time.perf_counter()
    print(f'Finished in {round(end - start, 6)} second(s)')
    print('')

    # ====================================================================================
    # ###############
    # Create Center Point
    # ###############
    start = time.perf_counter()

    if trjStartX > trjEndX:
        centX = trjStartX - ((trjStartX - trjEndX) / 2.0)
    else:
        centX = trjEndX - ((trjEndX - trjStartX) / 2.0)

    if trjStartY > trjEndY:
        centY = trjStartY - ((trjStartY - trjEndY) / 2.0)
    else:
        centY = trjEndY - ((trjEndY - trjStartY) / 2.0)

    print('Center Point (X/Y)        :({}, {})'.format(centX, centY))

    end = time.perf_counter()
    print(f'Finished in {round(end - start, 6)} second(s)')
    print('')

    # ====================================================================================
    # ###############
    # Create Angle
    # ###############
    start = time.perf_counter()

    distA = distB = distC = None
    blnSlope = 0

    if (trjStartX < trjEndX) and (trjStartY > trjEndY):
        blnSlope = 0
        trj3X = trjStartX
        trj3Y = trjEndY

        distA = math.sqrt(((trj3X - trjEndX) ** 2) + ((trj3Y - trjEndY) ** 2))
        distB = math.sqrt(((trj3X - trjStartX) ** 2) + ((trj3Y - trjStartY) ** 2))
        distC = math.sqrt(((trjStartX - trjEndX) ** 2) + ((trjStartY - trjEndY) ** 2))
    elif (trjStartX > trjEndX) and (trjStartY < trjEndY):
        blnSlope = 0
        trj3X = trjEndX
        trj3Y = trjStartY

        distA = math.sqrt(((trj3X - trjStartX) ** 2) + ((trj3Y - trjStartY) ** 2))
        distB = math.sqrt(((trj3X - trjEndX) ** 2) + ((trj3Y - trjEndY) ** 2))
        distC = math.sqrt(((trjEndX - trjStartX) ** 2) + ((trjEndY - trjStartY) ** 2))
    elif (trjStartX > trjEndX) and (trjStartY > trjEndY):
        blnSlope = 1
        trj3X = trjStartX
        trj3Y = trjEndY

        distA = math.sqrt(((trj3X - trjEndX) ** 2) + ((trj3Y - trjEndY) ** 2))
        distB = math.sqrt(((trj3X - trjStartX) ** 2) + ((trj3Y - trjStartY) ** 2))
        distC = math.sqrt(((trjStartX - trjEndX) ** 2) + ((trjStartY - trjEndY) ** 2))
    elif (trjStartX < trjEndX) and (trjStartY < trjEndY):
        blnSlope = 1
        trj3X = trjEndX
        trj3Y = trjStartY

        distA = math.sqrt(((trj3X - trjStartX) ** 2) + ((trj3Y - trjStartY) ** 2))
        distB = math.sqrt(((trj3X - trjEndX) ** 2) + ((trj3Y - trjEndY) ** 2))
        distC = math.sqrt(((trjEndX - trjStartX) ** 2) + ((trjEndY - trjStartY) ** 2))

    phi = math.acos((-(distB ** 2) + (distA ** 2) + (distC ** 2)) / (2 * distA * distC))


    theta = (90 - (phi * RAD_TO_DEG)) * DEG_TO_RAD

    print('theta                     :', theta)

    end = time.perf_counter()
    print(f'Finished in {round(end - start, 6)} second(s)')
    print('')

    # ====================================================================================
    # ###############
    # Create Lattice
    # ###############
    start = time.perf_counter()

    if blnSlope:
        fltRStartX, fltRStartY = rotate(centX, centY, trjStartX, trjStartY, theta)
        fltREndX, fltREndY = rotate(centX, centY, trjEndX, trjEndY, theta)
    else:
        fltRStartX, fltRStartY = rotate(centX, centY, trjStartX, trjStartY, -theta)
        fltREndX, fltREndY = rotate(centX, centY, trjEndX, trjEndY, -theta)

    # This is the count along the Y lines
    if fltRStartY > fltREndY:
        latYOffset = fltREndY
        cntYlat = (fltRStartY - fltREndY) / latBS3
    else:
        latYOffset = fltRStartY
        cntYlat = (fltREndY - fltRStartY) / latBS3

    latXOffset = fltRStartX - (flightWidth / 2)
    cntItemLat = int(cntYlat * cntX)

    # This code will create a lattice based on the planned flight path and then use the lattice to
    # identify suspect or bad areas in the LIDAR data.

    # The lattice will hold an integer identifying its state as follows:
    #   0 – neutral
    #   1 – good
    #   2 - bad
    #   3 – suspect
    #   4 - overlap
    #   5 - point outside of AOI, exceeded threshold, or z-axis to close to aircraft

    lattice = np.empty(cntItemLat * sys.getsizeof(int()))

    offsetXStart = latXOffset
    offsetY = latYOffset
    offsetX = None

    for cntI in range(cntItemLat):
        if cntI % int(flightWidth / latBS3) == 0:
            offsetX = offsetXStart
            offsetY += latBS3
        else:
            offsetX += latBS3

    print('offsetX                   :', offsetX)
    print('offsetY                   :', offsetY)

    end = time.perf_counter()
    print(f'Finished in {round(end - start, 6)} second(s)')
    print('')

    # ====================================================================================
    # ###############
    # Run Data Check
    # ###############

    start = time.perf_counter()

    cntItems = lasPointCount +1  # 5,715,149
    pointRadius = latBS3

    lattice = np.empty(int(cntItemLat * sys.getsizeof(int())), dtype='d')
    pArray = np.empty(shape=(cntItems, 2), dtype='d')

    counter = 0
    cntSuspect = 0
    cntBad = 0
    intFirst = 0
    intSecond = 0
    intThird = 0
    intFourth = 0
    pStart = 0
    pEnd = 0
    processes = []

    cores = 6
    div = int(lasPointCount / cores)
    print('div                       :', div)
    print('cntItems                  :',cntItems)

    queue = mp.Queue()

    for i in range(1, cores+1):
        if cores == 1:
            pStart = 0
            pEnd = lasPointCount
        else:
            if i < cores:
                if i == 1:
                    pStart = 0
                else:
                    pStart = pEnd
                pEnd = div * i
            else:
                pStart = pEnd
                pEnd = lasPointCount

        print(i, '- start/end             : {:7d} / {:7d}'.format(pStart, pEnd))

        proc = mp.Process(target=runDataCheck, args=(queue, i, pStart, pEnd))

        proc.start()
        processes.append(proc)

    print('')
    tmpArray = [queue.get() for _ in range(1, cores+1)]

    for proc in processes:
        proc.join()

    pArray = np.concatenate(tmpArray, axis=0)

    # ====================================================================================

    print('')
    print('len(tmpArray)             :', len(tmpArray))

    tmpA = 0
    for i in range(0,len(tmpArray)):
        print('  ', i, 'tmpArray             :', len(tmpArray[i]))
        tmpA += len(tmpArray[i])

    print('sum of tmpArray length    :', tmpA)
    print('')
    print('Point Count               :', lasPointCount)
    print('len(pArray)               :', len(pArray))
    print('len(lattice)              :', len(lattice))

    end = time.perf_counter()
    print('')
    print(f'Finished in {round(end - start, 2)} second(s)')


Flight Width              : 670
Lattice Box Size          : 25
Intensity Threshold       : 240
Range Distance            : 30
Point Reduction Tolerance : 2
sqrLatBS3                 : 5.0
latBS3                    : 1.6666666666666667
cntX                      : 402
Finished in 9.9e-05 second(s)

min x y z                 : 578968.1344510008 4020974.928830802 149.23310590628535
max x y z                 : 581876.0325148759 4025542.6993111866 362.79136129282415
Las First Time            : 414381.278874
LAS Last Time             : 414449.07153899997
Finished in 0.121173 second(s)

Position Count            : 1710

Average Z for aircraft from Trajectory file is 1357.7516952751982
Traj Start X              : 579276.754576971
Traj Start Y              : 4025429.8817389696
Traj End X                : 581513.9254636365
Traj End Y                : 4021200.2538259574
Finished in 0.004157 second(s)

Center Point (X/Y)        :(580395.3400203038, 4023315.0677824635)
Finished in 9e-06 second(s)

t